# Deteksi Tepi Plat Nomor Kendaraan


## Import Library


In [ ]:
# %pip install opencv-python matplotlib pillow scikit-image numpy pandas

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse
from skimage.measure import shannon_entropy

## 1. Import Dataset


### Menentukan direktori dataset


In [ ]:
data_dir = './src/dataset/raw data'

### Cek apakah direktori dataset ada


In [ ]:
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Dataset directory '{data_dir}' does not exist. Please check the path.")

### Load dataset


In [ ]:
def load_images_from_directory(directory):
    images = []
    filenames = []
    for filename in os.listdir(directory):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            filepath = os.path.join(directory, filename)
            image = cv2.imread(filepath)
            if image is not None:
                images.append(image)
                filenames.append(filename)
            else:
                print(f"Warning: Unable to load image {filename}.")
    return images, filenames

In [ ]:
# Load images
images, filenames = load_images_from_directory(data_dir)

# Check if images were loaded
if not images:
    raise ValueError("No images were found in the specified directory. Please add some images.")

### Menampilkan dataset


In [ ]:
def display_images(images, filenames, cols=3, title="Gambar"):
    rows = (len(images) + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    for i, ax in enumerate(axes.flat):
        if i < len(images):
            image_rgb = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)
            ax.imshow(image_rgb)
            ax.set_title(filenames[i])
            ax.axis('off')
        else:
            ax.axis('off')
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

In [ ]:
display_images(images, filenames, title="Original Images")

## 2. Preprocessing Image


### Folder output


In [ ]:
output_folder = "./src/dataset/preprocessed"
os.makedirs(output_folder, exist_ok=True)

### Konfigurasi kompresi dan reshape


In [ ]:
max_width = 1024  # Resolusi tujuan (width, height)
compression_quality = 100   # Kualitas JPEG (0-100, semakin tinggi semakin baik)

### Fungsi untuk kompresi dan analisis


In [ ]:
analysis_data = []
for filename in os.listdir(data_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(data_dir, filename)
        img = cv2.imread(file_path)

        # Dapatkan dimensi asli
        original_height, original_width = img.shape[:2]

        # Tentukan dimensi target dengan mempertahankan proporsi
        scaling_factor = max_width / original_width
        target_width = int(original_width * scaling_factor)
        target_height = int(original_height * scaling_factor)
        target_size = (target_width, target_height)

        # Resize citra asli sesuai proporsi
        resized_original = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

        # Kompresi citra
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, resized_original, [cv2.IMWRITE_JPEG_QUALITY, compression_quality])

        # Baca kembali citra hasil kompresi
        compressed_img = cv2.imread(output_path)

        # Hitung MSE, PSNR, dan Entropy
        mse_value = mse(resized_original, compressed_img)
        psnr_value = psnr(resized_original, compressed_img)
        entropy_value = shannon_entropy(compressed_img)

        # Simpan hasil analisis
        analysis_data.append({
            "Filename": filename,
            "Original Size (MB)": os.path.getsize(file_path) / (1024 * 1024),
            "Compressed Size (MB)": os.path.getsize(output_path) / (1024 * 1024),
            "MSE": mse_value,
            "PSNR": psnr_value,
            "Entropy": entropy_value,
            "Original Dimensions": f"{original_width}x{original_height}",
            "Resized Dimensions": f"{target_width}x{target_height}"
        })

### Tabel hasil analisis


In [ ]:
df_analysis = pd.DataFrame(analysis_data)
print(df_analysis)

### Simpan hasil analisis ke file CSV


In [ ]:
# df_analysis.to_csv("./src/analysis/preprocessed_analysis.csv", index=False)

In [ ]:
images, filenames = load_images_from_directory(output_folder)
display_images(images, filenames, title="Compressed Images")

### **Analisis Hasil**

1. **MSE dan PSNR**:

   - **MSE**: Nilai MSE berada di kisaran **0.617 hingga 1.183**, yang sangat rendah. Ini menunjukkan bahwa perubahan dari citra asli ke citra hasil kompresi sangat kecil.
   - **PSNR**: Nilai PSNR berkisar antara **47.4 dB hingga 50.2 dB**, yang merupakan kualitas sangat tinggi. Secara visual, perbedaan antara citra asli dan hasil kompresi hampir tidak terlihat.

2. **Entropy**:

   - Nilai entropy berkisar antara **7.29 hingga 7.95**, menunjukkan bahwa informasi dalam citra tetap kaya dan keragaman detail visual tetap terjaga setelah kompresi.

3. **Ukuran File**:
   - Ukuran file berkurang secara signifikan (dari rata-rata 3-4 MB ke kurang dari 1.1 MB), meskipun kualitas JPEG diatur ke **100**.
   - Resolusi citra juga disesuaikan menjadi **1024x1365**, yang mempertahankan proporsi asli.

---

### **Kelebihan**

- **Efisiensi**: Ukuran file yang lebih kecil membuat pemrosesan lebih cepat, terutama saat dataset besar.
- **Kualitas Terjaga**: Nilai PSNR di atas 47 dB memastikan bahwa kompresi tidak merusak kualitas citra secara signifikan.
- **Proporsi Terjaga**: Dimensi gambar mempertahankan rasio asli, yang sangat penting untuk analisis citra.
